<a href="https://colab.research.google.com/github/ayachiii/soturon/blob/main/export_mean_PLI%E8%87%AA%E4%BD%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# mean_PLI_人名.csvを出力するプログラム

In [3]:
!pip install heartpy pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.6 MB/s eta 0:00:00


In [1]:
import glob

In [5]:
import numpy as np
import pandas as pd
import heartpy as hp
from matplotlib import pyplot as plt
import os

PLI_?_人名_フェーズ.csvをファイルにアップロードする。(rest,boredom,flow,ultra,overload)

In [8]:
# --- 1. 設定項目 (Constantsの代わり) ---
# 解析したい被験者名
SUBJECT_NAME = "kumakura"
# 被験者ID (フォルダ名の頭についている数字。フォルダを使わない場合は無視されます)
#SUBJECT_ID = "0"

# 解析するフェーズのリスト
PHASE_LIST = ["rest", "boredom", "flow", "flow_ultra", "overload"]

# PLIデータのカラム名
COLUMNS_FOR_PLI = ['FpZ','FC3','FC4','FCz','O1','O2','Fp2']

# ファイルがある場所 (Colabのルートディレクトリ)
# フォルダの中にある場合は "/content/csv/0_kumakura/PLIdata" のように書き換えてください
DATA_DIR = "/content"

# 出力先パス
SAVE_PATH = os.path.join(DATA_DIR, f"mean_PLI_{SUBJECT_NAME}.csv")


# --- 2. ファイル検索用の関数 ---
def find_pli_file(subject, phase, base_dir):
    """
    指定されたフェーズのPLIファイルを探す関数
    ファイル名のパターン: PLI_?_{subject}_{phase}.csv
    """
    # ワイルドカード(*)を使って、数字部分が何であってもヒットするようにします
    search_pattern = os.path.join(base_dir, f"PLI_*_{subject}_{phase}.csv")

    files = glob.glob(search_pattern)
    if len(files) > 0:
        return files[0] # 最初に見つかったファイルを返す
    else:
        return None


# --- 3. メイン処理 ---

# 結果を貯めるDataFrame
df_mean = pd.DataFrame()

# ベースライン値の初期化
base_PLI_FC3 = 0
base_PLI_FC4 = 0
base_PLI_FCz = 0

# ★重要: まずRestデータを読み込んでおく（差分計算のため）
rest_file_path = find_pli_file(SUBJECT_NAME, "rest", DATA_DIR)

if rest_file_path is None:
    print(f"エラー: Restフェーズのファイルが見つかりません。")
    print(f"探した場所: {DATA_DIR}/PLI_*_{SUBJECT_NAME}_rest.csv")
else:
    print(f"Restファイル読み込み: {rest_file_path}")
    rest_df = pd.read_csv(rest_file_path)

    # 各フェーズのループ処理
    for phase in PHASE_LIST:

        # ファイルの検索
        path = find_pli_file(SUBJECT_NAME, phase, DATA_DIR)

        if path is None:
            print(f"スキップ: {phase} のファイルが見つかりません。")
            continue

        print("-------------------------------------------------")
        print(f"{SUBJECT_NAME}さんの{phase}フェーズ")

        df = pd.read_csv(path)

        if phase == "rest":
            # --- Restフェーズの処理 ---
            mean_values = []
            for column in COLUMNS_FOR_PLI:
                mean = df[column].mean()
                mean_values.append(mean)

                # 基準値(ベースライン)を保存
                if column == "FC3":
                    base_PLI_FC3 = mean
                if column == "FC4":
                    base_PLI_FC4 = mean
                if column == "FCz":
                    base_PLI_FCz = mean

            # DataFrame作成 (初期化)
            df_mean = pd.DataFrame([mean_values], columns=COLUMNS_FOR_PLI, index=[phase])

            # 平均カラムの追加
            df_mean['mean_all'] = df_mean.mean(axis=1)
            # 3ch平均 (FC3, FC4, FCz)
            df_mean['mean_3ch'] = (df_mean['FC3'] + df_mean['FC4'] + df_mean['FCz']) / 3

            print(df_mean)

        else:
            # --- その他のフェーズの処理 ---
            mean_values = []
            concentration = []
            max_concentrations = []

            for column in COLUMNS_FOR_PLI:
                mean = df[column].mean()
                mean_values.append(mean)

                # 集中度 (Concentration) の計算
                if column == "FC3":
                    # 平均値の差分
                    concentration_FC3 = base_PLI_FC3 - mean
                    concentration.append(concentration_FC3)

                    # 時系列データの差分の最大値
                    # (注: 行数が異なるとNaNが出る可能性がありますが、pandasがindexで合わせます)
                    concentrations_time_series = rest_df[[column]] - df[[column]]
                    max_val = concentrations_time_series.max()[column]

                    # print("-----------")
                    # print(max_val)
                    max_concentrations.append(max_val)

                if column == "FC4":
                    concentration_FC4 = base_PLI_FC4 - mean
                    concentration.append(concentration_FC4)

                    concentrations_time_series = rest_df[[column]] - df[[column]]
                    max_concentrations.append(concentrations_time_series.max()[column])

                if column == "FCz":
                    concentration_FCz = base_PLI_FCz - mean
                    concentration.append(concentration_FCz)

                    concentrations_time_series = rest_df[[column]] - df[[column]]
                    max_concentrations.append(concentrations_time_series.max()[column])

            # --- データの結合 ---
            print("-------------")
            # print(max_concentrations)
            # print(concentration)

            # 行データの作成
            row = np.concatenate([mean_values, concentration, max_concentrations])

            # カラム名の作成
            new_columns = np.concatenate([
                COLUMNS_FOR_PLI,
                ["concentration_FC3", "concentration_FC4", "concentration_FCz",
                 "max_concentration_FC3", "max_concentration_FC4", "max_concentration_FCz"]
            ])

            df_for_add = pd.DataFrame([row], columns=new_columns, index=[phase])

            # 集中度の平均計算
            concentration_mean = (df_for_add['concentration_FC3'] + df_for_add['concentration_FC4'] + df_for_add['concentration_FCz']) / 3
            max_concentration_mean = (df_for_add['max_concentration_FC3'] + df_for_add['max_concentration_FC4'] + df_for_add['max_concentration_FCz']) / 3

            df_for_add['concentration_mean'] = concentration_mean
            df_for_add['max_concentration_mean'] = max_concentration_mean

            # 全体平均・3ch平均の計算
            # mean_all計算時にconcentration系を含めないように注意が必要ですが、
            # 元コードでは df_for_add.mean(axis=1) しているので、全ての列の平均になります。
            # (もしPLIだけの平均が欲しい場合は修正が必要です)
            df_for_add['mean_all'] = df_for_add.mean(axis=1)

            mean_3ch = (df_for_add['FC3'] + df_for_add['FC4'] + df_for_add['FCz']) / 3
            df_for_add['mean_3ch'] = mean_3ch

            # 結果を結合
            df_mean = pd.concat([df_mean, df_for_add])

    # --- 4. 保存 ---
    print("\n--- 最終結果 ---")
    print(df_mean)

    df_mean.to_csv(SAVE_PATH)
    print(f"\nファイルが保存されました: {SAVE_PATH}")


Restファイル読み込み: /content/PLI_0_kumakura_rest.csv
-------------------------------------------------
kumakuraさんのrestフェーズ
           FpZ       FC3       FC4       FCz       O1        O2       Fp2  \
rest  0.479361  0.716581  0.540148  0.778225  0.34835  0.291814  0.403018   

      mean_all  mean_3ch  
rest  0.508214  0.678318  
-------------------------------------------------
kumakuraさんのboredomフェーズ
-------------
-------------------------------------------------
kumakuraさんのflowフェーズ
-------------
-------------------------------------------------
kumakuraさんのflow_ultraフェーズ
-------------
-------------------------------------------------
kumakuraさんのoverloadフェーズ
-------------

--- 最終結果 ---
                 FpZ       FC3       FC4       FCz        O1        O2  \
rest        0.479361  0.716581  0.540148  0.778225  0.348350  0.291814   
boredom     0.276021  0.446082  0.437729  0.560950  0.180391  0.162976   
flow        0.257495  0.434018  0.440532  0.380226  0.666523  0.531137   
flow_ultra  0.4